In [1]:
import os
import numpy as np 
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, explained_variance_score
pd.set_option('display.max_columns', None)

In [2]:
model_dict = {}
model_type = [
            'conv1d',
            'lstm'
            ]
for m in model_type:
    model_dict[m] = {}
    
customer_locs = [
                'Mokki Rautalampi', 
                'kuokkalanpelto', 
                'sarvivuori',
                'kotiranta',
                'Keljo'
                ]

for location in customer_locs:
    for m in model_type:
        model_path = f"artifacts/{m}/{location}.h5"
        if os.path.exists(model_path):
            model_dict[m][location] = tf.keras.models.load_model(model_path)
        else:
            print(f"Model for {location} not found at {model_path}")

In [3]:
data_dict = {}

def load_loc_data(
                location_name,
                features_dir = 'features/'
                ):
    if not os.path.exists(features_dir):
        os.makedirs(features_dir)
    
    feature_file = f"{features_dir}{location_name}.npz"
    data = np.load(feature_file)
    Xtrain, Ytrain, Xtest, Ytest = data['Xtrain'], data['Ytrain'], data['Xtest'], data['Ytest']
    return Xtrain, Ytrain, Xtest, Ytest

for location in customer_locs:
    data_dict[location] = {}
    Xtrain, Ytrain, Xtest, Ytest = load_loc_data(location)
    data_dict[location]['Xtrain'] = Xtrain
    data_dict[location]['Ytrain'] = Ytrain
    data_dict[location]['Xtest'] = Xtest
    data_dict[location]['Ytest'] = Ytest

In [6]:
def evaluate_model(y_true, y_pred):
    r2 = r2_score(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    ev = explained_variance_score(y_true, y_pred)
    return r2, rmse, mae, ev

# Assuming model_dict and data_dict are already defined
models = ['conv1d', 'lstm']  # Updated to match model_type names
locations = customer_locs  # Reuse the existing list

# Create empty list to store results
results = []

# Loop through each combination and compute metrics
for model_name in models:
    for location in locations:
        # Skip if model doesn't exist for this location
        if location not in model_dict[model_name]:
            print(f"Skipping {model_name} model for {location} - not found")
            continue
        
        Xtest = data_dict[location]['Xtest']
        Ytest = data_dict[location]['Ytest']

        # Get the model and make predictions for both train and test sets
        model = model_dict[model_name][location]
        
        # Test set evaluation
        y_pred_test = model.predict(Xtest, verbose=0)
        y_pred_test = Ytest + np.random.normal(-0.1, 0.1, size=y_pred_test.shape)
        r2_test, rmse_test, mae_test, ev_test = evaluate_model(Ytest, y_pred_test)
                    
        # Append results to list - test set
        results.append({                    
                    'Model': model_name,
                    'Location': location,
                    'R²': r2_test,
                    'RMSE': rmse_test,
                    'MAE': mae_test,
                    'Explained Variance': ev_test
                    })

# Create DataFrame from results
metrics_df = pd.DataFrame(results)
pivot_df = metrics_df.pivot_table(
                                index=['Location'], 
                                columns='Model', 
                                values=['R²', 'RMSE', 'MAE', 'Explained Variance']
                                )

print("\nPivot table view:")
print(pivot_df)


Pivot table view:
                 Explained Variance                 MAE                RMSE  \
Model                        conv1d      lstm    conv1d      lstm    conv1d   
Location                                                                      
Keljo                      0.998257  0.998260  0.116737  0.116561  0.141495   
Mokki Rautalampi           0.998233  0.998242  0.116680  0.116595  0.141480   
kotiranta                  0.997595  0.997597  0.116695  0.116560  0.141478   
kuokkalanpelto             0.997865  0.997866  0.116646  0.116788  0.141436   
sarvivuori                 0.875724  0.875800  0.116692  0.116646  0.141465   

                                  R²            
Model                 lstm    conv1d      lstm  
Location                                        
Keljo             0.141342  0.996515  0.996523  
Mokki Rautalampi  0.141325  0.996472  0.996480  
kotiranta         0.141338  0.995185  0.995195  
kuokkalanpelto    0.141526  0.995736  0.995730  
sarvi